In [ ]:
import shap
import sys
sys.path.append("D:/ASOML/SNOCONE")
import numpy as np
import pandas as pd
import os
from CNN_benchmarks import*
from CNN_memoryOptimization import*
from CNN_preProcessing import*
from CNN_benchmarks import*
from CNN_modelArchitectureBlocks import*
import tensorflow as tf
import gc
import psutil
import matplotlib.pyplot as plt
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import threading
final_activation = 'relu'

# establish parameters
Domain = "Rockies"
basin_name = "Conejos"
WorkspaceBase = f"D:/ASOML/{Domain}/"
ModelOutputs = f"{WorkspaceBase}/modelOutputs/BasinSpecifics/"
time_code = "20250721_132214"
model_interation = f"{basin_name}_{time_code}"
feature_Listcsv = f"{ModelOutputs}/{Domain}_{basin_name}_model_featureList_summary.csv"
best_weights = ModelOutputs + f"/{model_interation}/best_weights_{model_interation}.h5"
start_year = 2022
end_year = 2024
shap_output = f"{ModelOutputs}/{model_interation}/shap_results/"
architecture = "FCN_SWE"
shapeChecks = "N"

X = np.load(f"D:/ASOML/Rockies/saved_stacks/{basin_name}/X_stack.npy")
y = np.load(f"D:/ASOML/Rockies/saved_stacks/{basin_name}/y_stack.npy")

# For feature names (a list of strings), use allow_pickle=True
featureNames = np.load(f"D:/ASOML/Rockies/saved_stacks/{basin_name}/feature_names.npy", allow_pickle=True)
featNo = len(featureNames)

# Run the optimized SHAP analysis
results = run_shap(weights_path=best_weights, 
                   X_sample=X, 
                   feature_names=featureNames, 
                   featNo=featNo, 
                   architecture=architecture, 
                   final_activation="relu", 
                   output_dir=shap_output)

Loading model...
Model loaded successfully. No compilation needed for SHAP analysis.
Creating SHAP with gradient explainer...
Calculating SHAP values...


C:\Users\etyrr\anaconda3\envs\aso-dl\lib\site-packages\keras\backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
